# Over the Rainbow : il ponte quantistico tra immaginazione e realtà

Il nostro progetto... (breve descrizione) procediamo ad una base d'esempio dell'algortimo vedendo le fasi di : 
* Setup 
* Inizializzazione
* Caricamento del dataset
* Spiegazione del QuantumEnhancedEnsambleClassifier

### Iniziamo dal setup 
Richiamiamo tutte le librerie ed effettuiamo gli import necessari 

In [2]:
# Setup
%pip install "qiskit"
%pip install "qiskit[visualization]~=2.1.0" "qiskit-serverless~=0.24.0" "qiskit-ibm-catalog~=0.8.0" "scikit-learn==1.5.2" "pandas>=2.0.0,<3.0.0" "imbalanced-learn~=0.12.3"


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 667.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 1.1 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: qiskit-serverless
    Found existing installation: qiskit-serverless 0.26.1
    Uninstalling qiskit-serverless-0.26.1:
      Successfully uninstalled qiskit-serverless-0.26.1
  Attempting uninstall: qiskit-ibm-catalog
    Found existing installation: qiskit-ibm-catalog 0.10.0
    Uninstalling qiskit-ibm-catalog-0.10.0:
      Successfully uninstalled qiskit-ibm-catalog-0.10.0

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Importazioni

import sys
import os
import tarfile
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from qiskit_serverless import IBMServerlessClient, QiskitFunction
from typing import Tuple
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier

# Inizializzazione della funzione 'Singularity Machine Learning - Classification'

Si inizia dal predisporre l'account su cui eseguire gli IBM Qiskit Serverless client per recuperare la funzione dal Multiverse Computing workspace. NOTA : bisogna disporre di un account con piano IBM Quantum per poter inserire la API key ed eseguire il codice. 

Caricando la funzione 'Singularity Machine Learning - Classification' si ha accesso a un classificatore quantum-enhanced che può essere usato per modelli ibridi classici quantistici. Questa funzione permette di unire componenti classici e quantistici per migliorare il tranining, ottimizzazine e la predizione dei modelli.  

In [ ]:
print("Using IBM Qiskit Serverless...\n")

# Caricamento dell'IBM Serverless Client
your_api_key = "cancellareEdInserireAPIKeydellAccountIBMQuantum"
your_crn = "cancellareEdInserireCRNdell'istanzaIBMQuantum"

client = IBMServerlessClient(
    channel="ibm_quantum_platform",
    token=your_api_key,
    instance=your_crn,
)
# Dovrebbe essere mostrata una lista di Funzioni Qiskit disponibili

# Caricamento della funzione Multiverse Singularity

catalog.load("multiverse/singularity")
singularity = client.get("multiverse/singularity")
client.list()

# Definizione delle variabili statiche

RANDOM_STATE: int = 123
TRAIN_PATH = "data/grid_stability/train.csv"
TEST_PATH = "data/grid_stability/test.csv"

# Definizione delle Helper Functions

Per  semplificare il workflow, definiamo una serie di funzioni di supporto (_helper functions_) utilizzate per la gestione dei dati. Queste funzioni consentono un'interfaccia fluida tra l'elaborazione dei dati locali e l'esecuzione remota con Qiskit Serverless.

In [ ]:
def load_data(data_path: str) -> Tuple[np.ndarray, np.ndarray]:
    """Load data from the given path to X and y arrays."""
    df: pd.DataFrame = pd.read_csv(data_path)
    return df.iloc[:, :-1].values, df.iloc[:, -1].values


def make_tarfile(file_path, tar_file_name):
    """Create a tar file from the given file."""
    with tarfile.open(tar_file_name, "w") as tar:
        tar.add(file_path, arcname=os.path.basename(file_path))


def upload_data(name: str, data: np.ndarray, client: IBMServerlessClient, function: QiskitFunction):
    """Save the data to a file, create a tar file, upload it, and remove the files."""
    np.save(name, data)
    make_tarfile(name, f"{name}.tar")
    client.file_upload(f"{name}.tar", function)

    
def evaluate_predictions(predictions, y_true, start_time=None, end_time=None):
    accuracy = accuracy_score(y_true, predictions)
    precision = precision_score(y_true, predictions)
    recall = recall_score(y_true, predictions)
    f1 = f1_score(y_true, predictions)
    if start_time is not None and end_time is not None:
        print("Time taken (s):", end_time - start_time)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1:", f1)
    return accuracy, precision, recall, f1

# Caricamento del dataset
Successivamente, carichiamo e pre-elaboriamo il dataset utilizzato per addestrare e valutare il nostro classificatore potenziato quantisticamente (quantum-enhanced classifier). Il dataset consiste in campioni etichettati relativi a un compito di classificazione degli artefatti di rendering, dove l'obiettivo è predire se un frame è accettabile o richiede re-rendering basandosi sui dati di output del motore di denoising.

Iniziamo leggendo i dati di addestramento (training) e di test (che rappresentano feature vector estratti da migliaia di frame) da file CSV e poi suddividiamo ulteriormente il set di addestramento per creare un set di validazione (validation set). Per affrontare lo squilibrio di classe (class imbalance)—dato che i frame con artefatti sono rari—applichiamo l'oversampling casuale (random over-sampling) per aumentare i campioni della classe minoritaria ("Artefatto/Rendere"), assicurando che il classificatore non sia distorto verso l'etichetta dominante ("Pulito e Accettabile").

Una volta preparati, i dataset vengono serializzati come array NumPy e caricati nell'ambiente Qiskit Serverless. Ciò consente alla funzione remota 'Singularity Machine Learning - Classification' di accedere e operare sui dati durante l'esecuzione distribuita per l'ottimizzazione dell'ensemble.

In [ ]:
# NOTA : ovviamente tutto il codice che segue è solo un esempio

# Caricamento e upload dei dati
X_train, y_train = load_data(TRAIN_PATH)
X_test, y_test = load_data(TEST_PATH)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=RANDOM_STATE
)
# Bilanciamento del dataset attraverso l'over-sampling della classe positiva
ros = RandomOverSampler(random_state=RANDOM_STATE)
X_train, y_train = ros.fit_resample(X_train, y_train)

In [ ]:
print("-- Uploading VFX Artifacts Data --")
# Caricamento dei feature vector di addestramento (X) e delle etichette (y)
upload_data("render_qc_X_train.npy", X_train, client, singularity)
upload_data("render_qc_y_train.npy", y_train, client, singularity)
# Caricamento dei feature vector per il set di test
upload_data("render_qc_X_test.npy", X_test, client, singularity)
print("VFX Data uploaded to Qiskit Serverless!\n")

# AdaBoostClassifier

Ora che i dati sono pronti, iniziamo utilizzando l'AdaBoostClassifier, un metodo ensemble ben consolidato nel machine learning classico. AdaBoost funziona combinando più apprenditori deboli (weak learners) in un unico classificatore forte, migliorando iterativamente le prestazioni concentrandosi maggiormente sui campioni difficili da classificare.

Come esempio si può addestrare il modello AdaBoost utilizzando il set di addestramento bilanciato e si può valutare sul set di test. Questo fornisce una solida baseline di confronto con il classificatore potenziato quantisticamente (Quantum Enhanced Ensamble Classifier) che utilizzeremo in seguito. Il numero di estimator è fissato a 75 per permettere all'ensemble di generalizzare bene senza incorrere in overfitting.

Dopo l'addestramento, generiamo le predizioni sul set di test e calcoliamo le metriche di performance chiave—accuratezza, precisione, richiamo (recall) e punteggio F1—per valutare quanto bene il modello si comporta in questo compito di classificazione degli artefatti di rendering.

In [ ]:
classifier = AdaBoostClassifier(n_estimators=75, random_state=RANDOM_STATE)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)

In [ ]:
evaluate_predictions(predictions, y_test)

# QuantumEnhancedEnsembleClassifier

### Introduzione 

Questo è il core dell'algoritmo! Il QuantumEnhancedEnsembleClassifier è un modello di machine learning ibrido proveniente dalla funzione 'Singularity Machine Learning - Classification' di Multiverse Computing. Esso combina i metodi ensemble classici come il boosting e il bagging—con algoritmi di ottimizzazione quantistica come il QAOA.

A differenza degli ensemble classici, il cui addestramento diventa sempre più costoso all'aumentare del numero di learner, questo classificatore quantistico dimostra proprietà di scalabilità favorevoli: il tempo di addestramento rimane relativamente stabile all'aumentare del numero di learner (e di conseguenza dei qubit). Ciò lo rende particolarmente adatto per problemi che richiedono ensemble ampi come la classificazione robusta di artefatti VFX—dove i modelli classici tendono a soccombere a costi di ottimizzazione che scalano esponenzialmente.

Inoltre, a differenza dei modelli quantistici tradizionali come i QSVM, che sono spesso vincolati dalla dimensione del dataset, questo classificatore è progettato per operare indipendentemente dal numero di punti dati e feature limitato primariamente dall'infrastruttura hardware disponibile. Questo lo rende ideale per l'elaborazione di dataset ad alta dimensionalità e su larga scala, come i feature vector estratti da migliaia di frame di rendering.

Man mano che l'hardware quantistico scala, anche l'accuratezza e le prestazioni di questo modello si evolveranno, offrendo una traiettoria convincente per il futuro vantaggio quantistico in applicazioni reali. In questo notebook, mostreremo una parte di codice finalizzata poi all'utilizzo sia in simulazione classica che in esecuzione quantistica reale, mettendone in luce il potenziale e la flessibilità.

![Diagram](how_it_works.jpeg)

---

## Comprendere i Parametri di Ottimizzazione (Optimization Parameters)

Per usare effettivamente il `QuantumEnhancedEnsembleClassifier`, è importante comprendere il ruolo di alcuni parametri chiave che influenzano il model training e optimization. Per ulteriori informazioni è disponibile la [documentatazione](https://docs.quantum.ibm.com/guides/multiverse-computing-singularity) ufficiale. 

ALcuni dei parametri chiave sono :

- **`regularization`**  
  Questo parametro controlla la complessità dell'ensemble finale. Un valore più alto penalizza combinazioni di learner (apprenditori) ampie o complesse, promuovendo modelli più semplici e riducendo il rischio di overfitting. 

- **`num_solutions`**  
   Definisce il numero di configurazioni ensemble candidate che l'ottimizzatore valuta. Valori più elevati portano a ricerche più esaustive e possono scoprire ensemble con prestazioni migliori, ma a costo di un aumento del tempo di esecuzione (runtime). A scopo dimostrativo, valori intorno a `100,000` bilanciano l'esplorazione e la performance. 

- **`simulator`**  
  Impostare questo parametro su `True` abilita l'ottimizzazione classica più veloce e ben si adatta alla sperimentazione rapida, ma non beneficia dei vantaggi di scalabilità del calcolo quantistico. Impostarlo su `False` attiva l'ottimizzazione quantistica tramite QAOA, l'ideale per ensemble più grandi. 

- **`classical_optimizer_options`**  
  Queste opzioni definiscono il comportamento dell'ottimizzatore classico sottostante utilizzato nel QAOA. Ad esempio, `{"maxiter": 10}` limita il numero di iterazioni di ottimizzazione—utile per brevi dimostrazioni, ma la convergenza di solito migliora con `60+` iterazioni nelle applicazioni reali.

Questi parametri danno la flessibilità di bilanciare velocità, accuratezza e uso delle risorse di calcolo. Da qui in poi si può andare avanti e sperimentare, sia con configurazioni classiche che quantistiche, per osservare questi _trade-offs_ in azione."